In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd


# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers.csv"

### Edit cell below to change phone number count

In [4]:
# set phone number count
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

In [5]:
# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

#shuffle phone number lists
random.shuffle(gojek_phone_numbers)
random.shuffle(grab_phone_numbers)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)

# Define classes for the necessary for the algorithm

In [6]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename

                


    
        

# Create psi client class

In [7]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1):
        
        # instantiate classes required for generation and selection of numbers for psi algorithm
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.0.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.0.csv"
        
        # create files
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator
        
        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp
        
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            other_party_set = self.receive_data("my_self_encrypted_set") # read the other party's self-encrypted set
            other_party_set_int = []
            for element_string in other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
        
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
        
        
    def get_intersection(self):
        # get intersection 
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set")  # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element_string in my_encrypted_set: 
            my_encrypted_set_int.append(int(element_string))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            self.common_values.append(self.my_set[index])
            
        # update csv file for communication
        self.send_data(self.common_values, "common_values")
    
    def send_data(self, data_to_send, column_name):
        # send data means writing to file (my file)
        self.my_dict[column_name] = data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (other party's file)
        
        data = self.get_other_party_data()[column_name].to_list()
        cleaned_data = [x for x in data if type(x) is not float] # remove NaN
        print(f"number of elements in {self.other_party_name};{column_name}: {len(cleaned_data)}.")
        return cleaned_data
        
    def get_my_data(self):
        # read my file as dataframe
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)


    


# Initialize context

### Edit cell below to change key size

In [8]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

In [9]:
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_phone_numbers, p, factors_pminus1)
gojek = Client("gojek", "grab", key_size, gojek_phone_numbers, p, factors_pminus1)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_4084/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [10]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Status after step 1

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,7501383186321213515512277498775635912732518619...,3565755568280639079050051854599706700190867755...
1,8092639001204638790178908996283896242788771916...,5930688136427591374346485570270456146366904370...
2,2689155155339598219508325922471219247576193922...,7322471674394226755197759817505338232518241311...
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...
4,4339504937591743225701797258110083974962550965...,3864964095203405700528472210556810964594978309...
...,...,...
86,NaN,1016971668148270186279384292396393102811041724...
87,NaN,3683371775181761031500661295222456997304332645...
88,NaN,8119100922771943066177300284568368128847410924...
89,NaN,5011913842513548422134267800545138946266586949...


## Step 2: encrypt hashed set with own private key

In [12]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_4084/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [13]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,7501383186321213515512277498775635912732518619...,3565755568280639079050051854599706700190867755...,3017291284986515492778098813721155161591952601...,3887263166179224738730136680749886821413097280...
1,8092639001204638790178908996283896242788771916...,5930688136427591374346485570270456146366904370...,1081649207370106213507891725294665719431331235...,1216203050125353433229484904559730955009881016...
2,2689155155339598219508325922471219247576193922...,7322471674394226755197759817505338232518241311...,6298664978795527917109659911730699270697015690...,7453510698174534936570062695571679445017333989...
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...,1242797027128547468917797762116922827552447538...,3155318262388753944519743994913465854113208725...
4,4339504937591743225701797258110083974962550965...,3864964095203405700528472210556810964594978309...,3612121026094085852350615476171566029234906650...,3359143703772521526378000553175552005613480014...
...,...,...,...,...
86,NaN,1016971668148270186279384292396393102811041724...,NaN,3815831469935040716599988930767383145470110602...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1033193090860010399038425505459555475546999812...
88,NaN,8119100922771943066177300284568368128847410924...,NaN,6392392605027572248620254593336824001495880988...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,1064600754078673328463152313282611080502450355...


### Files' status after step 2 (value of variables in files)

In [14]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek

gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,3017291284986515492778098813721155161591952601...,NaN,NaN
1,1081649207370106213507891725294665719431331235...,NaN,NaN
2,6298664978795527917109659911730699270697015690...,NaN,NaN
3,1242797027128547468917797762116922827552447538...,NaN,NaN
4,3612121026094085852350615476171566029234906650...,NaN,NaN
...,...,...,...
56,4711161467347410402810786098217245787498877100...,NaN,NaN
57,6026323275526406155829260655371975356324263080...,NaN,NaN
58,1116575315952960676796926163864151292895223403...,NaN,NaN
59,2880818441787784510566685442924504695588236787...,NaN,NaN


In [15]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,3887263166179224738730136680749886821413097280...,NaN,NaN
1,1216203050125353433229484904559730955009881016...,NaN,NaN
2,7453510698174534936570062695571679445017333989...,NaN,NaN
3,3155318262388753944519743994913465854113208725...,NaN,NaN
4,3359143703772521526378000553175552005613480014...,NaN,NaN
...,...,...,...
86,3815831469935040716599988930767383145470110602...,NaN,NaN
87,1033193090860010399038425505459555475546999812...,NaN,NaN
88,6392392605027572248620254593336824001495880988...,NaN,NaN
89,1064600754078673328463152313282611080502450355...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key

In [16]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 61.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_4084/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 91.


### Clients' status after step 3

In [17]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,7501383186321213515512277498775635912732518619...,3565755568280639079050051854599706700190867755...,3017291284986515492778098813721155161591952601...,3887263166179224738730136680749886821413097280...,4034741481474501500586224492992265857855868805...,1044558065657365644992324892994623061399557810...
1,8092639001204638790178908996283896242788771916...,5930688136427591374346485570270456146366904370...,1081649207370106213507891725294665719431331235...,1216203050125353433229484904559730955009881016...,1265115483379193035752861966276711174788736568...,2684130299531042579541403689009222691558724340...
2,2689155155339598219508325922471219247576193922...,7322471674394226755197759817505338232518241311...,6298664978795527917109659911730699270697015690...,7453510698174534936570062695571679445017333989...,7461863638408409229011347223213047395424825867...,1249096855550425448343765077415730414825147553...
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...,1242797027128547468917797762116922827552447538...,3155318262388753944519743994913465854113208725...,7822551484776719562432625831858730484300959172...,1135200633982743504472449496376918927640762621...
4,4339504937591743225701797258110083974962550965...,3864964095203405700528472210556810964594978309...,3612121026094085852350615476171566029234906650...,3359143703772521526378000553175552005613480014...,3375347033338047920348039234153590233001518972...,8648013112800789717471095650211287707231976431...
...,...,...,...,...,...,...
86,NaN,1016971668148270186279384292396393102811041724...,NaN,3815831469935040716599988930767383145470110602...,NaN,6606160460699540507741986712410395597237604991...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1033193090860010399038425505459555475546999812...,NaN,1102288439799464902464124902048193396394878731...
88,NaN,8119100922771943066177300284568368128847410924...,NaN,6392392605027572248620254593336824001495880988...,NaN,3427516327870296777122642842067449462979951307...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,1064600754078673328463152313282611080502450355...,NaN,9548520961031594503899288387085972247988692745...


### Files' status after step 3

In [18]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek

gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,3017291284986515492778098813721155161591952601...,1044558065657365644992324892994623061399557810...,NaN
1,1081649207370106213507891725294665719431331235...,2684130299531042579541403689009222691558724340...,NaN
2,6298664978795527917109659911730699270697015690...,1249096855550425448343765077415730414825147553...,NaN
3,1242797027128547468917797762116922827552447538...,1135200633982743504472449496376918927640762621...,NaN
4,3612121026094085852350615476171566029234906650...,8648013112800789717471095650211287707231976431...,NaN
...,...,...,...
86,NaN,6606160460699540507741986712410395597237604991...,NaN
87,NaN,1102288439799464902464124902048193396394878731...,NaN
88,NaN,3427516327870296777122642842067449462979951307...,NaN
89,NaN,9548520961031594503899288387085972247988692745...,NaN


In [19]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,3887263166179224738730136680749886821413097280...,4034741481474501500586224492992265857855868805...,NaN
1,1216203050125353433229484904559730955009881016...,1265115483379193035752861966276711174788736568...,NaN
2,7453510698174534936570062695571679445017333989...,7461863638408409229011347223213047395424825867...,NaN
3,3155318262388753944519743994913465854113208725...,7822551484776719562432625831858730484300959172...,NaN
4,3359143703772521526378000553175552005613480014...,3375347033338047920348039234153590233001518972...,NaN
...,...,...,...
86,3815831469935040716599988930767383145470110602...,NaN,NaN
87,1033193090860010399038425505459555475546999812...,NaN,NaN
88,6392392605027572248620254593336824001495880988...,NaN,NaN
89,1064600754078673328463152313282611080502450355...,NaN,NaN


## Step 4: find intersection

In [20]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 91.
number of elements in grab;other_party_encrypted_set: 61.


### Clients' status after step 4:

In [21]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,7501383186321213515512277498775635912732518619...,3565755568280639079050051854599706700190867755...,3017291284986515492778098813721155161591952601...,3887263166179224738730136680749886821413097280...,4034741481474501500586224492992265857855868805...,1044558065657365644992324892994623061399557810...,95521626.0,95521626.0
1,8092639001204638790178908996283896242788771916...,5930688136427591374346485570270456146366904370...,1081649207370106213507891725294665719431331235...,1216203050125353433229484904559730955009881016...,1265115483379193035752861966276711174788736568...,2684130299531042579541403689009222691558724340...,99173089.0,99173089.0
2,2689155155339598219508325922471219247576193922...,7322471674394226755197759817505338232518241311...,6298664978795527917109659911730699270697015690...,7453510698174534936570062695571679445017333989...,7461863638408409229011347223213047395424825867...,1249096855550425448343765077415730414825147553...,96485172.0,96485172.0
3,4271190063387899149690355220761209149002412790...,1017141269956754882131908776753968058318200171...,1242797027128547468917797762116922827552447538...,3155318262388753944519743994913465854113208725...,7822551484776719562432625831858730484300959172...,1135200633982743504472449496376918927640762621...,94391128.0,94391128.0
4,4339504937591743225701797258110083974962550965...,3864964095203405700528472210556810964594978309...,3612121026094085852350615476171566029234906650...,3359143703772521526378000553175552005613480014...,3375347033338047920348039234153590233001518972...,8648013112800789717471095650211287707231976431...,89312048.0,96192082.0
...,...,...,...,...,...,...,...,...
86,NaN,1016971668148270186279384292396393102811041724...,NaN,3815831469935040716599988930767383145470110602...,NaN,6606160460699540507741986712410395597237604991...,NaN,NaN
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1033193090860010399038425505459555475546999812...,NaN,1102288439799464902464124902048193396394878731...,NaN,NaN
88,NaN,8119100922771943066177300284568368128847410924...,NaN,6392392605027572248620254593336824001495880988...,NaN,3427516327870296777122642842067449462979951307...,NaN,NaN
89,NaN,5011913842513548422134267800545138946266586949...,NaN,1064600754078673328463152313282611080502450355...,NaN,9548520961031594503899288387085972247988692745...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [22]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek

gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,3017291284986515492778098813721155161591952601...,1044558065657365644992324892994623061399557810...,95521626.0
1,1081649207370106213507891725294665719431331235...,2684130299531042579541403689009222691558724340...,99173089.0
2,6298664978795527917109659911730699270697015690...,1249096855550425448343765077415730414825147553...,96485172.0
3,1242797027128547468917797762116922827552447538...,1135200633982743504472449496376918927640762621...,94391128.0
4,3612121026094085852350615476171566029234906650...,8648013112800789717471095650211287707231976431...,89312048.0
...,...,...,...
86,NaN,6606160460699540507741986712410395597237604991...,NaN
87,NaN,1102288439799464902464124902048193396394878731...,NaN
88,NaN,3427516327870296777122642842067449462979951307...,NaN
89,NaN,9548520961031594503899288387085972247988692745...,NaN


In [23]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,3887263166179224738730136680749886821413097280...,4034741481474501500586224492992265857855868805...,95521626.0
1,1216203050125353433229484904559730955009881016...,1265115483379193035752861966276711174788736568...,99173089.0
2,7453510698174534936570062695571679445017333989...,7461863638408409229011347223213047395424825867...,96485172.0
3,3155318262388753944519743994913465854113208725...,7822551484776719562432625831858730484300959172...,94391128.0
4,3359143703772521526378000553175552005613480014...,3375347033338047920348039234153590233001518972...,96192082.0
...,...,...,...
86,3815831469935040716599988930767383145470110602...,NaN,NaN
87,1033193090860010399038425505459555475546999812...,NaN,NaN
88,6392392605027572248620254593336824001495880988...,NaN,NaN
89,1064600754078673328463152313282611080502450355...,NaN,NaN


# Check results

In [24]:
# get intersection found by the two parties
grab_data = grab.get_my_data()
gojek_data = gojek.get_my_data()
gojek_found_intersection = gojek_data["common_values"].tolist()
grab_found_intersection = grab_data["common_values"].tolist()

# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])).dropna(how = 'all')
df

,actual,gojek,grab
0,80497694.0,80497694.0,80497694.0
1,81093373.0,81093373.0,81093373.0
2,86915509.0,86915509.0,86915509.0
3,89312048.0,89312048.0,89312048.0
4,94391128.0,94391128.0,94391128.0
5,95521626.0,95521626.0,95521626.0
6,96192082.0,96192082.0,96192082.0
7,96485172.0,96485172.0,96485172.0
8,99173089.0,99173089.0,99173089.0
9,99397525.0,99397525.0,99397525.0
